In [26]:
import backtrader as bt
from datetime import datetime

In [39]:
class MovingAverageStrategy(bt.Strategy):

    # Moving average parameters
    params = (('pfast',20),('pslow',50),)
    
    def __init__(self):
        # get the data we have provided
        # self.close_data = self.data.close
        self.close_data = self.datas[0].close
        
        # this is where we create the indicators
        self.fast_sma = bt.indicators.MovingAverageSimple(self.close_data,
                                                         period=self.params.pfast)
        self.slow_sma = bt.indicators.MovingAverageSimple(self.close_data,
                                                         period=self.params.pslow)        
        

    def next(self):
        
        # This function will be called automatically after
        # a new data point (stock price) is available
        # print("%s - %s" % (self.data.datetime.date(0), self.close_data[0]))        
        
        # we have to check whether we have already opened a long position
        if not self.position:
            # we can open a long position if needed
            if self.fast_sma[0] > self.slow_sma[0] \
                    and self.fast_sma[-1] < self.slow_sma[-1]:
                print('BUY')
                self.buy()
                
        else:
            # check whether to close the long position
            if self.fast_sma[0] < self.slow_sma[0] \
                    and self.fast_sma[-1] > self.slow_sma[-1]:
                print('CLOSE')
                self.close()

In [40]:
if __name__ == "__main__":
    
    cerebro = bt.Cerebro()
    
    # add data to the core
    stock_data = bt.feeds.YahooFinanceData(dataname='AMZN',
                                    fromdate=datetime(2010, 1, 1),
                                    todate=datetime(2020, 1, 1))
    
    # we have to add the data to Cerebro
    cerebro.adddata(stock_data)
    cerebro.addstrategy(MovingAverageStrategy)
    
    cerebro.addobserver(bt.observers.Value)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0)
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    
    #commission fees - set 0.1%
    cerebro.broker.setcommission(0.01)
    
    # run the strategy
    cerebro.broker.set_cash(30000)
    print('Initial capital: $%.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    
    # evaluate the results
    print('Sharpe ratio: %.2f' % results[0].analyzers.sharperatio.
          get_analysis()['sharperatio'])
    print('Return: %.2f%%' % results[0].analyzers.returns.
         get_analysis()['rnorm100'])
    print('Max Drawdown: %.2f' % results[0].analyzers.drawdown.
         get_analysis()['max']['drawdown'])
    print('Capital: $%.2f' % cerebro.broker.getvalue())

Initial capital: $30000.00
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
Sharpe ratio: 0.44
Return: 0.28%
Max Drawdown: 1.77
Capital: $30838.42
